# 통계 스터디 Quiz

- 일시: 2023.12.29(금) 19:00
- 범위: 3장
- 출제: 이우진

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import pickle

In [2]:
with open('chapter_3.pkl', 'rb') as file:
    chapter_3 = pickle.load(file)

quiz_2_data = chapter_3['quiz_2']
quiz_3_data = chapter_3['quiz_3']

---

### 1.

연 2회 시행되고 만점이 100점인 자격증 시험이 있다.

- 1회차 시험의 성적 자료는 평균 70점, 표준편차 10점의 정규분포를 따른다.
- 2회차 시험의 성적 자료는 평균 60점, 표준편차 15점의 정규분포를 따른다.

#### (1)

Z점수는 해당 회차의 모집단에 대해 정규화한 점수를 의미한다. 각각의 회차에서 만점자의 Z점수를 구하시오.

In [3]:
answers = {
    '1회자 만점자 Z점수': (100 - 70) / 10,
    '2회자 만점자 Z점수': (100 - 60) / 15
}

for key, value in answers.items():
    print(f"{key}: {round(value, 3)}")

1회자 만점자 Z점수: 3.0
2회자 만점자 Z점수: 2.667


$$z = \frac{x - \mu}{\sigma}$$

위와 같이 모집단의 평균, 표준편차를 이용하여 정규화할 수 있다.

#### (2)

각 회차에서 60점 이상을 기록할 확률을 구하시오.

In [4]:
answers = {
    '1회차 확률': 1 - stats.norm.cdf((60 - 70) / 10),
    '2회차 확률': 1 - stats.norm.cdf((60 - 60) / 15)
}

for key, value in answers.items():
    print(f"{key}: {round(value, 3)}")

1회차 확률: 0.841
2회차 확률: 0.5


다음의 과정을 거쳐서 해결할 수 있다.
- 기준점인 60점을 정규화하여 Z점수를 구한다.
- Z점수를 넣어 CDF를 구하면, 해당 회차에서 60점 **이하**를 기록할 확률을 얻게 된다. 이때 scipy의 ```stats.norm.cdf()```를 이용할 수 있다.
- 1에서 그 값을 빼주면, 60점 이상을 기록할 확률이 된다.

#### (3)

상위 20%에 해당하는 응시자만 합격이라고 할 때, 각 회차의 합격 기준 점수를 구하시오.

In [5]:
answers = {
    '1회차 합격 기준 점수': stats.norm.ppf(0.8) * 10 + 70,
    '2회차 합격 기준 점수': stats.norm.ppf(0.8) * 15 + 60
}

for key, value in answers.items():
    print(f"{key}: {round(value, 2)}점")

1회차 합격 기준 점수: 78.42점
2회차 합격 기준 점수: 72.62점


(2)에서 진행한 과정을 반대로 한다고 생각하면 된다.
- 상위 20%는 곧 하위 80%와 같다. 따라서 CDF가 $1 - 0.2 = 0.8$이 되는 지점을 찾아야 한다.
- CDF의 역함수인 PPF를 이용하여, CDF가 0.8이 될 때의 Z점수를 구한다. 이때 scipy의 ```stats.norm.ppf()```를 이용할 수 있다.
- 해당 Z점수를 역정규화하여 원점수를 구한다.

---

### 2.

어떤 IT 기업에서 채용을 위한 코딩테스트를 진행했다. 이 기업의 인사 팀은 응시자의 MBTI 중 '사고(T)'의 비율과 코딩테스트 점수 간의 관계를 분석하는 프로젝트를 시행하고자 한다.

In [6]:
quiz_2_data

,사고(T) 비율,점수
0,0.051380,58.027610
1,0.284721,58.694412
2,0.500000,72.000000
3,0.500000,69.000000
4,0.101264,70.025271
...,...,...
295,0.500000,71.000000
296,0.500000,88.000000
297,0.500000,89.000000
298,0.500000,71.000000


#### (1)

응시자의 사고 비율과 코딩테스트 점수 사이의 공분산을 구하시오.

In [7]:
pd.DataFrame(np.cov([quiz_2_data.iloc[:, 0], quiz_2_data.iloc[:, 1]]))

,0,1
0,0.045128,0.942034
1,0.942034,103.636218


numpy의 ```np.cov()```를 이용하면 위와 같이 공분산 행렬(Covariation Matrix)을 얻을 수 있다. 이때 [0, 1] 또는 [1, 0]이 공분산에 해당한다. [0, 0]은 사고(T) 비율 컬럼의 분산, [1, 1]은 점수 컬럼의 분산을 의미한다.

In [8]:
answer = np.cov([quiz_2_data.iloc[:, 0], quiz_2_data.iloc[:, 1]])[0, 1]
print(f"공분산: {np.round(answer, 3)}")

공분산: 0.942


#### (2)

응시자의 사고 비율과 코딩테스트 점수 사이의 피어슨 상관계수를 구하시오.

In [9]:
pd.DataFrame(np.corrcoef([quiz_2_data.iloc[:, 0], quiz_2_data.iloc[:, 1]]))

,0,1
0,1.0000,0.4356
1,0.4356,1.0000


numpy의 ```np.corrcoef()``` 함수를 이용하면 위와 같이 상관계수 행렬을 얻을 수 있다. 이를 통해 각 컬럼들 사이의 관계를 알아볼 수 있다. 사고(T) 비율 컬럼과 점수 컬럼 사이의 피어슨 상관계수는 [0, 1] 또는 [1, 0]으로 인덱싱하면 된다.

In [10]:
answer = np.corrcoef([quiz_2_data.iloc[:, 0], quiz_2_data.iloc[:, 1]])[0, 1]
print(f"피어슨 상관계수: {np.round(answer, 3)}")

피어슨 상관계수: 0.436


In [ ]:
from scipy.stats import pearsonr

pearsonr(quiz_2_data['~~~'], quiz_2_data['~~~~'])[0]

---

### 3.

IT 취준생 100명에게 Python과 Java 사용 여부에 대한 설문을 진행한 결과가 다음과 같다.

In [11]:
quiz_3_data

,Python,Java
0,X,O
1,X,O
2,X,O
3,O,X
4,X,O
...,...,...
95,O,O
96,O,O
97,X,O
98,X,O


#### (1)

설문에 참여한 취준생 중 임의로 선택한 한 명이 Python을 사용하면서 Java는 사용하지 않을 확률을 구하시오.

In [12]:
python_o_java_x = quiz_3_data[
    (quiz_3_data['Python'] == 'O') & (quiz_3_data['Java'] == 'X')
].count()[0]
answer = python_o_java_x / quiz_3_data.count()[0]
print(f"확률: {np.round(answer, 3)}")

확률: 0.4


문제를 다음과 같은 사건 A와 사건 B의 결합확률로 이해할 수 있다.
- 사건 A: Python을 사용한다.
- 사건 B: Java를 사용하지 않는다.

pandas의 조건 인덱싱을 활용하면 ${A}\cap{B}$를 쉽게 구할 수 있으며, 이를 통해 확률을 도출할 수 있다.

#### (2)

설문에 참여한 취준생 중 임의로 선택한 한 명이 Python을 사용하는 사람일 때, Java를 사용하지 않을 확률을 구하시오.

In [13]:
python_o = quiz_3_data[(quiz_3_data['Python'] == 'O')].count()[0]
answer = python_o_java_x / python_o
print(f"확률: {np.round(answer, 3)}")

확률: 0.548


이번 문제는 사건 A, 사건 B에 대한 조건부확률로 이해할 수 있다.

$$P(B|A) = \frac{P(A \cap B)}{P(A)}$$

이때 앞서 (1)에서 구한 $P(A \cap B)$를 활용하면 된다.

---

### Appendix

In [95]:
# 2번 데이터 생성
np.random.seed(826)
t_ratio = 0.5 + np.random.rand(300) * 0.45 * np.random.randint(-1, 2, 300)
coding_score = 65 + t_ratio * 20 + np.random.randint(-15, 16, 300)
quiz_2_data = pd.DataFrame({
    '사고(T) 비율': t_ratio,
    '점수': coding_score
})

In [96]:
# 3번 데이터 생성
np.random.seed(826)
quiz_3 = [['O', 'O']] * 33 + [['O', 'X']] * 40 + [['X', 'O']] * 24 + [['X', 'X']] * 3
np.random.shuffle(quiz_3)
quiz_3_data = pd.DataFrame(quiz_3, columns=['Python', 'Java'])

In [104]:
# 데이터 합쳐서 저장
chapter_3 = {
    'quiz_2': quiz_2_data,
    'quiz_3': quiz_3_data
}

with open('chapter_3.pkl', 'wb') as file:
    pickle.dump(chapter_3, file)